# Reprojecting GOES JPG and PNG Images to Mercator in Python with GDAL
**Lance Berc**

2019-12-03

## Synopsis
Imagery from NASA/NOAA weather satellites is readily available from many sources, but the raw data needs to be reprojected from the satellite's viewpoint to something more familiar for the images to "make sense".

In my case I want to use the Mercator projection because (a) it's what we're used to from ship's navagational charts and grammer school wall maps, and (b) it allows overlaying some of the weather charts available from NOAA's Ocean Prediction Center. It should be easy to convert this example to other projections such as Lambert Conformal Conic (LCC) used by US aeronautical charts and some NOAA weather models like the NAM and HRRR.

Most scientific satellite imagery comes in NetCDF format containing georeferencing information which allows the image to be accurately mapped to the Earth. The simple images we use don't have this so we derive it from satellite parameters.

Definitions for GOES-16 and GOES-17 are given; it should be easy to extend to other geostationary satellites such as the European Meteosats and Japan's Himawari-8.

In Mode 6 (the current operational mode) GOES satellites produce full-disk images every 10 minutes and CONUS images every 5 minutes. They can also produce images from two higher-resolution Mesoscale regions but because the locations move they aren't handled here.

## References

GOES projection is defined in section 5.1.2.7 of

https://www.goes-r.gov/users/docs/PUG-L1b-vol3.pdf

Meteosat projection is defined in section 4.4.3.2 of

https://www.cgms-info.org/documents/cgms-lrit-hrit-global-specification-(v2-8-of-30-oct-2013).pdf

https://www.cgms-info.org/documents/pdf_cgmf_03.pdf (older version)

GDAL/OGR comes from OSGeo, the Open Source Geospatial Foundation. It works with the Proj projection manipulation software.

https://www.osgeo.org/

https://gdal.org/

https://proj.org/

Many of the projection's are defined in the EPSG's Well Known Text (WKT) format:

https://www.epsg.org

NASA and EUMETSAT publish great reference material; it can be a bit dense:

https://www.goes-r.gov/resources/docs.html

https://www.eumetsat.int/website/home/Data/TechnicalDocuments/index.html

##### Nota Bene
This example reprojects a raster image into another raster image using GDAL; it's also possible to do this with Matplotlib+Cartopy. Using Cartopy with pcolormesh to reproject a 5000x3000 1k CONUS image took 37 seconds on my home machine - in a loop processing multiple images it occupies all 8GB of RAM; with GDAL a similar operation took about a second and has a working set less than 500MB. GDAL can process a GOES 5424x5424 2k full-disk image in 6 seconds - I haven't waited for pcolormesh to finish.

## Example 1 - GOES-17 CONUS (PACUS)

In [1]:
#!/usr/bin/python
import numpy as np
from PIL import Image
from osgeo import gdal
import os

The specifications for the GOES satellites fall into two categories. First are those about the orbit and perspective above the GRS-80 ellipsoid. Second are per-resolution items that have the scan angle, raster size, and positioning of the upper-left corner.

"1k" and "2k" represent the per-pixel resolutions of the images. There are both higher- (".5k") and lower ("4k" and "10k") resolutions defined for various instruments.

In [2]:
goes16 = { # GOES-16, aka GOES-R, GOES-EAST
    "p_height": 35786023.0,           # perspective height from the ellipsoid
    "height": 42164160.0,             # from center of the earth
    "longitude": -75.0,
    "sweep_axis": 'x',
    "semi_major": 6378137.0,          # GRS-80 ellipsoid
    "semi_minor": 6356752.31414,      # GRS-80 ellipsoid
    "flattening": 298.257222096,
    "eccentricity": 0.0818191910435,
    # The other resolution (.5k, 4k, etc) can be added here
    "1k": {
        "resolution": 0.000028,       # radians per pixel
        "FD": {
            "x_offset": -0.151858,    # radians from nadir
            "y_offset":  0.151858,
            "shape": (10848, 10848),  # pixels in image
        },
        "CONUS": {
            "x_offset": -0.101346,
            "y_offset":  0.128226,
            "shape": (5000, 3000),
        }
    },
    "2k": {
        "resolution": 0.000056,
        "FD": {
            "x_offset": -0.151844,
            "y_offset":  0.151844,
            "shape": (5424, 5424),
        },
        "CONUS": {
            "x_offset": -0.101332,
            "y_offset":  0.128212,
            "shape": (2500, 1500),
        }
    }
}

goes17 = { # GOES-17, aka GOES-S, GOES-WEST
    "p_height": 35786023.0,
    "height": 42164160.0,
    "longitude": -137.0,
    "sweep_axis": 'x',
    "semi_major": 6378137.0,
    "semi_minor": 6356752.31414,
    "flattening": 298.257222096,
    "eccentricity": 0.0818191910435,
    "1k": {
        "resolution": 0.000028,
        "FD": {
            "x_offset": -0.151858,
            "y_offset":  0.151858,
            "shape": (10848, 10848),
        },
        "CONUS": { # aka PACUS because it's mostly Pacific Ocean
            "x_offset": -0.069986,
            "y_offset":  0.128226,
            "shape": (5000, 3000),
        }
    },
    "2k": {
        "resolution": 0.000056,
        "FD": {
            "x_offset": -0.151844,
            "y_offset":  0.151844,
            "shape": (5424, 5424),
        },
        "CONUS": {
            "x_offset": -0.069972,
            "y_offset":  0.128212,
            "shape": (2500, 1500),
        }
    },
}

The EPSG definition of Mercator doesn't allow longitudes that extend
past -180 or 180 which makes working in the Pacific difficult. Define
our own, plus one with the centralized on the anti-meridian to allow
working with GOES-17 (which crosses the anti-meridian) continuous.

In [3]:
proj_mercator =      "+proj=merc +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +over"
proj_anti_mercator = "+proj=merc +k=1 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs +over +lon_0=-180"

Define source & destination files and the output dimensions and coverage area.
This interest area isn't exactly "square" wrt Mercator, but is close.

In [4]:
sourcefn = "./GOES-17_CONUS.jpg"
destfn = "./GOES-17_Mercator.jpg"
(w, h) = (1280, 720) # 16:9 aspect ratio HD-sized (not FullHD which is 1920x1280)
interestArea = [-152.0, 30.0, -110.0, 50.0] # lat/long of ll, ur corners

GDAL affine transformation

Raster location through geotransform (affine) array `[upperleftx, scalex, skewx, upperlefty, skewy, scaley]`

`upperleftx`, `upperlefty`: distance from center (satellite nadir) in meters

`scalex`, `scaley`: size of one pixel of raster projection in meters

`skewx`, `skewy` are zero because the images are not rotated wrt the equator

scaley is negative because scan GOES order is from top (positive) towards bottom (negative). This might have to change for other satellites.

In [10]:
s = goes17
res = s["1k"]
sector = res["CONUS"]
upper_left_x = sector['x_offset'] * s['p_height']
upper_left_y = sector['y_offset'] * s['p_height']
resolution_m = res['resolution'] * s['p_height']

geotransform = [upper_left_x, resolution_m, 0, upper_left_y, 0, -resolution_m]

Proj and EPSG descriptions from NASA specs. The warp option SOURCE_EXTRA is about what to do when the projection is off-earth. Turning on multithreading doesn't seem to improve speed on my Windows7 setup.

In [6]:
proj = "+proj=geos +lon_0=%f +h=%f +a=%f +b=%f +f=%f +units=m +no_defs -ellps=GRS80 +sweep=%s +over" % (
        s['longitude'], s['p_height'], s['semi_major'], s['semi_minor'], 1/s['flattening'], s['sweep_axis'])
WKT = 'PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",%f,%f]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]\
,PROJECTION["Geostationary_Satellite"],PARAMETER["central_meridian",%f],PARAMETER["satellite_height",%f],PARAMETER["false_easting",0],PARAMETER["fal\
se_northing",0],UNIT["Meter",1],EXTENSION["PROJ4","%s"]]' % (s['semi_major'], s['flattening'], s['longitude'], s['p_height'], proj)

warpOptions = gdal.WarpOptions(
    format="MEM",
    width=w,
    height=h,
    outputBoundsSRS="EPSG:4326", # WGS84 - Allows use of lat/lon outputBounds
    outputBounds=interestArea,
    dstSRS=proj_anti_mercator, # GOES-17 full-disk crosses the anti-meridian
    # Setting GDAL_PAM_ENABLED should suppress sidecar emission, but it doesn't
    # warpOptions=["SOURCE_EXTRA=1000", "GDAL_PAM_ENABLED=FALSE", "GDAL_PAM_ENABLED=NO"],
    warpOptions=["SOURCE_EXTRA=500"], # Magic from The Internet
    multithread = True, # Not sure if this buys anything on my setup
)


Do the warp, convert to a PIL image, save the image to a file, then clean up.

In [7]:
src = gdal.Open(sourcefn, gdal.GA_ReadOnly)
src.SetGeoTransform(geotransform)
src.SetProjection(WKT)

dst = gdal.Warp('', src, options=warpOptions)
if not dst:
    print("Warp failed %s" % (fn))
    img = None
else:
    dsta = dst.ReadAsArray() # Array shape is [band, row, col]
    arr = dsta.transpose(1, 2, 0) # Virtually change the shape to [row, col, band]
    img = Image.fromarray(arr, 'RGB')
    img.save(destfn, "JPEG")

# Clean up - this is important if running in a loop
src = None
dst = None
dsta = None
arr = None

# A side effect of setting dst to None is that the sidecar is emmitted when dst is "closed"
sidecar = "%s.aux.xml" % (sourcefn)
if os.path.isfile(sidecar): # Would be neat to figure out how to supress sidecar emission
    os.unlink(sidecar)

### GOES-17 CONUS (PACUS) Input
![input image](./GOES-17_CONUS.jpg)
### GOES-17 CONUS (PACUS) Output
![input image](./GOES-17_MERCATOR.jpg)

## Example 2 - GOES-16 Full Disk
The same as above, but with GOES-16 Full Disk input at 2k resolution.

In [8]:
sourcefn = "./GOES-16_FD.jpg"
destfn = "./GOES-16_Mercator.jpg"
(w, h) = (1280, 720) # 16:9 aspect ratio HD-sized (not FullHD which is 1920x1280)
interestArea = [-90, 18.5, -57, 35] # lat/long of ll, ur corners
s = goes16
res = s["2k"]
sector = res["FD"]

Everything else is the same as the above example.

In [9]:
upper_left_x = sector['x_offset'] * s['p_height']
upper_left_y = sector['y_offset'] * s['p_height']
resolution_m = res['resolution'] * s['p_height']

geotransform = [upper_left_x, resolution_m, 0, upper_left_y, 0, -resolution_m]
proj = "+proj=geos +lon_0=%f +h=%f +a=%f +b=%f +f=%f +units=m +no_defs -ellps=GRS80 +sweep=%s +over" % (
        s['longitude'], s['p_height'], s['semi_major'], s['semi_minor'], 1/s['flattening'], s['sweep_axis'])
WKT = 'PROJCS["unnamed",GEOGCS["unnamed ellipse",DATUM["unknown",SPHEROID["unnamed",%f,%f]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433]]\
,PROJECTION["Geostationary_Satellite"],PARAMETER["central_meridian",%f],PARAMETER["satellite_height",%f],PARAMETER["false_easting",0],PARAMETER["fal\
se_northing",0],UNIT["Meter",1],EXTENSION["PROJ4","%s"]]' % (s['semi_major'], s['flattening'], s['longitude'], s['p_height'], proj)

warpOptions = gdal.WarpOptions(
    format="MEM",
    width=w,
    height=h,
    outputBoundsSRS="EPSG:4326", # WGS84 - Allows use of lat/lon outputBounds
    outputBounds=interestArea,
    dstSRS=proj_anti_mercator, # GOES-17 full-disk crosses the anti-meridian
    # Setting GDAL_PAM_ENABLED should suppress sidecar emission, but it doesn't
    # warpOptions=["SOURCE_EXTRA=1000", "GDAL_PAM_ENABLED=FALSE", "GDAL_PAM_ENABLED=NO"],
    warpOptions=["SOURCE_EXTRA=500"], # Magic from The Internet
    multithread = True, # Not sure if this buys anything on my setup
)
src = gdal.Open(sourcefn, gdal.GA_ReadOnly)
src.SetGeoTransform(geotransform)
src.SetProjection(WKT)

dst = gdal.Warp('', src, options=warpOptions)
if not dst:
    print("Warp failed %s" % (fn))
    img = None
else:
    dsta = dst.ReadAsArray() # Array shape is [band, row, col]
    arr = dsta.transpose(1, 2, 0) # Virtually change the shape to [row, col, band]
    img = Image.fromarray(arr, 'RGB')
    img.save(destfn, "JPEG")

# Clean up - this is important if running in a loop
src = None
dst = None
dsta = None
arr = None

# A side effect of setting dst to None is that the sidecar is emmitted when dst is "closed"
sidecar = "%s.aux.xml" % (sourcefn)
if os.path.isfile(sidecar): # Would be neat to figure out how to supress sidecar emission
    os.unlink(sidecar)

### GOES-16 Full Disk Input
![input image](./GOES-16_FD.jpg)
### GOES-16 Full Disk Output
![input image](./GOES-16_MERCATOR.jpg)